In [24]:
from dotenv import load_dotenv
load_dotenv()  # .env 파일 로드

from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector


chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])


examples = [
    {"question": "일본", "answer": """
     수도 : 도쿄
     인구: 9,273,000
     면적: 2,191 km²
     언어: 일본어
     통화: 일본 엔 (JPY)
     """},
    {"question": "프랑스", "answer": """
     수도 : 파리
     인구: 2,140,526
     면적: 105.4 km²
     언어: 프랑스어
     통화: 유로 (EUR)
     공식 이름: 파리
     """},
    {"question": "영국", "answer": """
     수도 : 런던
     인구: 8,982,000
     면적: 1,572 km²
     언어: 영어
     통화: 파운드 스털링 (GBP)
     공식 이름: 런던
     """},
]



class RandomExampleSelector(BaseExampleSelector):
  def __init__(self, examples):
    self.examples = examples

  def add_example(self, example):
    self.examples.append(example)

  def select_examples(self, input_variables):
    from random import choice
    return [choice(self.examples)]



example_prompt = PromptTemplate.from_template("""
    Human: {question}을 수도를 갖고있는 나라의 정보를 알려줘 AI: {answer}
""")

example_selector = RandomExampleSelector(
  examples=examples,
)

prompt = FewShotPromptTemplate(
  example_prompt=example_prompt,
  example_selector=example_selector,
  suffix="Human: {question}을 수도를 갖고있는 나라의 정보를 알려줘",
  input_variables=["question"]
)

prompt.format(question="일본")

'\n    Human: 프랑스을 수도를 갖고있는 나라의 정보를 알려줘 AI: \n     수도 : 파리\n     인구: 2,140,526\n     면적: 105.4 km²\n     언어: 프랑스어\n     통화: 유로 (EUR)\n     공식 이름: 파리\n     \n\n\nHuman: 일본을 수도를 갖고있는 나라의 정보를 알려줘'